In [1]:
import scipy.stats as ss
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

##### 1 task

##### 2 task

In [30]:
def sample_size(gamma, alpha, delta):
    error = delta + 1
    n = 1
    while error >= delta:
        z_quantile = ss.norm.ppf(1 - alpha)
        phi_z_quantile = ss.norm.pdf(z_quantile)
        error = (gamma/(6*np.sqrt(n)))*(2*z_quantile**2+1)*phi_z_quantile
        n+=1
    return n-1

result = sample_size(2, 0.025, 0.025 * 0.1)
print(result)

##### 3 task